# Care Copilot MVP

## Key Features
- User prompt -> chart creation
- Automatic (LLM generated) or manual chart type and names (currently only bar charts and pie charts)

## Future Improvements
- Better object creation workflow
- Error reporting features
- create a more fluid flow - i.e feedback between different layers of the program

In [1]:
import sys
sys.path
if 'f:\\Users\\hassa\\carecognitics_backend' not in sys.path:
    sys.path.append('f:\\Users\\hassa\\carecognitics_backend')

In [2]:
from supersetapiclient.client import SupersetAPIClient
from security.nl_dml_security_layer import NL_SECURITY_LAYER
from llm.sql_generator import invoke_full_chain
from visualization_recommendation_system.visual_recommender import visual_recommender 



## manual dashbaord buliding using charts workflow

In [3]:
sc=SupersetAPIClient(base_url='http://localhost:8088',
                     username='admin',
                     password='admin')

### create new dashboard (optional)

In [4]:
# initiate new dashboard to add future charts to
sc.create_dashboard(name='Global Track sales Deep Dive')

### NL input

In [5]:
input="list the top 5 countries with the greatest amount of spending on tracks"
cleaner =NL_SECURITY_LAYER(input)
protection_output = cleaner.protect_input()

if not protection_output:
    final_output=invoke_full_chain(input)
else:
    print(protection_output)

In [6]:
sql_query, dataset_name, chart_type, viz_type, slice_name = final_output

In [7]:
print(f" query: {sql_query}\n------------------------------")
print(f" dataset_name: {dataset_name}\n------------------------------")
print(f"chart type: {chart_type}\n------------------------------")
print(f"viz_type: {viz_type}\n------------------------------")
print(f"slice_name: {slice_name}\n------------------------------")

 query: SELECT billing_country, SUM(total) AS total_spending FROM invoice GROUP BY billing_country ORDER BY total_spending DESC LIMIT 5;
------------------------------
 dataset_name: Top_Spending_Countries
------------------------------
chart type: Bar Chart
------------------------------
viz_type: echarts_timeseries_bar
------------------------------
slice_name: Top 5 Spending Countries
------------------------------


### create new dataset from LLM generated SQL

In [8]:
sc.create_dataset(sql=sql_query, table_name=dataset_name)


dataset has been created with name Top_Spending_Countries


### choose a chart type manually (optional)

In [9]:
vr = visual_recommender()

In [10]:
feature_dict=vr.parse_sql(sql=sql_query)
feature_dict

{'agg': 'SUM', 'agg_by': 'total', 'groupby': 'billing_country'}

In [9]:
# viz_type=vr.generate_visual_recommendations(sql=sql_query, return_type='best')

### create the chart

In [11]:
sc.create_chart(slice_name=slice_name,
                       viz_type=viz_type,
                       dashboard_ids=None)

### repeat for different inputs - QUESTION 2

In [12]:
input="fetch the data to make a pie chart for most top 5 most spending states in the USA"
cleaner =NL_SECURITY_LAYER(input)
protection_output = cleaner.protect_input()

if not protection_output:
    final_output_1=invoke_full_chain(input)
    final_output_1
else:
    print(protection_output)

('SELECT billing_state AS "State", SUM(total) AS "Total Spending" FROM invoice WHERE billing_country = \'USA\' GROUP BY billing_state ORDER BY SUM(total) DESC LIMIT 5;', 'usa_spending_by_state', 'Pie Chart', 'pie', 'The Big Spenders: Top 5 Spending States in the USA')


In [14]:
sql_query_1, dataset_name_1, chart_type_1, viz_type_1, slice_name_1 = final_output_1
print(f" query: {sql_query_1}\n------------------------------")
print(f" dataset_name: {dataset_name_1}\n------------------------------")
print(f"chart type: {chart_type_1}\n------------------------------")
print(f"viz_type: {viz_type_1}\n------------------------------")
print(f"slice_name: {slice_name_1}\n------------------------------")

 query: SELECT billing_state AS "State", SUM(total) AS "Total Spending" FROM invoice WHERE billing_country = 'USA' GROUP BY billing_state ORDER BY SUM(total) DESC LIMIT 5;
------------------------------
 dataset_name: usa_spending_by_state
------------------------------
chart type: Pie Chart
------------------------------
viz_type: pie
------------------------------
slice_name: The Big Spenders: Top 5 Spending States in the USA
------------------------------


In [16]:
sc.create_dataset(sql=sql_query_1, table_name=dataset_name_1)

dataset has been created with name usa_spending_by_state


In [13]:
vr = visual_recommender()

In [14]:
feature_dict=vr.parse_sql(sql_query_1)
feature_dict

{'agg': 'COUNT', 'agg_by': '*', 'groupby': 'billing_city;'}

In [15]:
# viz_type=vr.generate_visual_recommendations(sql=sql_query_1, return_type='best')

In [18]:
sc.create_chart(slice_name=slice_name_1,
                       viz_type=viz_type_1,
                       dashboard_ids=None)

### Question 3

In [27]:
input="fetch the data to make a bar chart for the most popular genres in the USA"
cleaner =NL_SECURITY_LAYER(input)
protection_output = cleaner.protect_input()

if not protection_output:
    final_output_2=invoke_full_chain(input)
    final_output_2
else:
    print(protection_output)

In [28]:
sql_query_2, dataset_name_2, chart_type_2, viz_type_2, slice_name_2 = final_output_2
print(f" query: {sql_query_2}\n------------------------------")
print(f" dataset_name: {dataset_name_2}\n------------------------------")
print(f"chart type: {chart_type_2}\n------------------------------")
print(f"viz_type: {viz_type_2}\n------------------------------")
print(f"slice_name: {slice_name_2}\n------------------------------")

 query: SELECT g.name AS Genre, COUNT(*) AS Track_Count FROM track AS t JOIN invoice_line AS il ON t.track_id = il.track_id JOIN invoice AS i ON il.invoice_id = i.invoice_id JOIN genre AS g ON t.genre_id = g.genre_id WHERE i.billing_country = 'USA' GROUP BY g.name ORDER BY Track_Count DESC LIMIT 3;
------------------------------
 dataset_name: Music_Genre_Popularity_USA
------------------------------
chart type: Bar Chart
------------------------------
viz_type: echarts_timeseries_bar
------------------------------
slice_name: Popularity of Music Genres in the USA
------------------------------


In [29]:
sc.create_dataset(sql=sql_query_2, table_name=dataset_name_2)

dataset has been created with name Music_Genre_Popularity_USA


In [31]:
sc.create_chart(slice_name=slice_name_2,
                       viz_type=viz_type_2,
                       dashboard_ids=[20])

### Question 4

In [8]:
input="make me a pie chart on top spenders on rock music in CA, only return two columns, customer_name and total_spent"
cleaner =NL_SECURITY_LAYER(input)
protection_output = cleaner.protect_input()

if not protection_output:
    final_output_3=invoke_full_chain(input)
    final_output_3
else:
    print(protection_output)

In [9]:
sql_query_3, dataset_name_3, chart_type_3, viz_type_3, slice_name_3 = final_output_3
print(f" query: {sql_query_3}\n------------------------------")
print(f" dataset_name: {dataset_name_3}\n------------------------------")
print(f"chart type: {chart_type_3}\n------------------------------")
print(f"viz_type: {viz_type_3}\n------------------------------")
print(f"slice_name: {slice_name_3}\n------------------------------")

 query: SELECT c.first_name || ' ' || c.last_name AS customer_name, SUM(i.total) AS total_spent FROM customer c JOIN invoice i ON c.customer_id = i.customer_id JOIN invoice_line il ON i.invoice_id = il.invoice_id JOIN track t ON il.track_id = t.track_id JOIN genre g ON t.genre_id = g.genre_id WHERE c.state = 'CA' AND g.name = 'Rock' GROUP BY c.customer_id, customer_name ORDER BY total_spent DESC LIMIT 5;
------------------------------
 dataset_name: ca_rock_music_spenders
------------------------------
chart type: Pie Chart
------------------------------
viz_type: pie
------------------------------
slice_name: Top 5 Spenders on Rock Music in California
------------------------------


In [11]:
sc.create_dataset(sql=sql_query_3, table_name=dataset_name_3)

dataset has been created with name ca_rock_music_spenders


In [12]:
sc.create_chart(slice_name=slice_name_3,
                       viz_type=viz_type_3,
                       dashboard_ids=[20])